In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [95.2 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [983 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports In

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...|2015-08-31 00:00:00|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...| 

In [5]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R27ZP1F1CD0C3Y|          5|            0|          0|   N|
| RJ7RSBCHUDNNE|          5|            0|          0|   N|
|R1HVYBSKLQJI5S|          5|            0|          0|   N|
|R2HAXF0IIYQBIR|          1|            0|          0|   N|
| RNYLPX611NB7Q|          5|            0|          0|   N|
|R3ALA9XXMBEDZR|          5|            0|          0|   N|
|R3R8PHAVJFTPDF|          5|            0|          0|   N|
|R18WWEK8OIXE30|          5|            0|          0|   N|
|R1EGUNQON2J277|          1|            0|          0|   N|
|R21Z4M4L98CPU2|          5|            0|          0|   N|
| R6JH7A117FHFA|          5|            0|          0|   N|
|R1XZHS8M1GCGI7|          5|            1|          1|   N|
|R1DAI0N03SKRJN|          5|            1|          1|   N|
|R2F6SKZOEYQRU3|          5|            

In [7]:
# Filter DataFrame for total_votes >= 20:
df_1 = df.filter(df.total_votes >= 20)
df_1.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...|2015-08-26 00:00:00|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...| 

In [8]:
# Filter df_1 for helpful_votes / total_votes >= 50%:
df_2 = df_1.filter((df_1.helpful_votes / df_1.total_votes) >= 0.5)
df_2.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...|2015-08-26 00:00:00|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...| 

In [9]:
# Filter d_2 for vine =='Y' (paid):
df_paid = df_2.filter(df_2.vine =='Y')
df_paid.show()

+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
|marketplace|customer_id|review_id|product_id|product_parent|product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|review_headline|review_body|review_date|
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+
+-----------+-----------+---------+----------+--------------+-------------+----------------+-----------+-------------+-----------+----+-----------------+---------------+-----------+-----------+



In [10]:
# Filter d_2 for vine =='N' (unpaid):
df_unpaid = df_2.filter(df_2.vine =='N')
df_unpaid.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   18888920|R36PQ9D8L2AAOH|B00JDQJVF2|     255898874|Amazon.com Gift C...|       Gift Card|          5|           30|         32|   N|                Y|          Five Stars|Love this adorabl...|2015-08-26 00:00:00|
|         US|    4765280|R23GHBE86II0SK|B007V6ETDK|     924812503|Amazon eGift Card...| 

In [11]:
#Total number of reviews:
df_unpaid.count()

355

In [19]:
#Total number of 5-star reviews:
df_unpaid_5stars = df_unpaid.filter(df_unpaid.star_rating==5)
df_unpaid_5stars.count()

90

In [20]:
# Percentage of 5-star reviews for unpaid:
percentage_unpaid = (df_unpaid_5stars.count() / df_unpaid.count()) * 100
percentage_unpaid

25.352112676056336